In [149]:
# Libraries needed (pandas is not standard and must be installed in Python)
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import numpy as np
import geopandas as gpd

# Insert your own client ID here
with open("../ignore_me/client_id.txt") as f: client_id = f.read().strip()

